# CKD Plan Designer Tutorial

## Install CKDPlanDesigner Package

In [1]:
! pip install -e project_code/

Obtaining file:///Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code
  Preparing metadata (setup.py) ... done
  Attempting uninstall: CKDPlanDesigner
    Found existing installation: CKDPlanDesigner 0.0.0
    Uninstalling CKDPlanDesigner-0.0.0:
      Successfully uninstalled CKDPlanDesigner-0.0.0
  Running setup.py develop for CKDPlanDesigner


## Load Data
- This tutorial uses the freely available synthea example dataset
    - Download it here: https://synthetichealth.github.io/synthea-sample-data/downloads/synthea_sample_data_csv_apr2020.zip
    - unzip it to SYNTHEA_PATH
 

In [49]:
import pandas as pd
import os

from CKDPlanDesigner.utils import data_preprocessing as pre

%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
SYNTHEA_PATH = '/Users/nicholasdeveau/projects/bmi210/bmi210Project_data/synthea_data/'

#### Here we build our patient dataframe
- This contains a vecor for each patient containing the required information for plan designing 

In [51]:
patients_df = pre.preprocess_synthea_data(SYNTHEA_PATH)

/Users/nicholasdeveau/projects/bmi210/bmi210Project/project_code/src/CKDPlanDesigner/utils/data_preprocessing.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sel_cond_df['VALUE'] = 1


## Working With a Patient

In [52]:
from CKDPlanDesigner.models import patients
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
# patient_config = {'patient_id': 123,2
#                   'eGFR': 70,
#                   'depression': True
#                  }


# select the first patient and convert feature vector to a dictionary
# we pass this dictionary to the patient class on instantiation
patient1_config = patients_df.to_dict(orient='index')[0]
p1 = patients.Patient(**patient1_config)

patient2_config = patients_df.to_dict(orient='index')[40]
p2 = patients.Patient(**patient2_config)

#### Look at patient 1's charecteristics

In [54]:
p1.patient_config

{'patient_id': 'c8403116-d069-4130-aa02-39954fc971b5',
 't2d': True,
 'depression': False,
 'bmi': False,
 'hypertension': False,
 'smoking': False,
 'dob': '1967-03-26',
 'race': 'white',
 'ethnicity': 'nonhispanic',
 'gender': 'F',
 'zip': nan,
 'eGFR': 66.9}

In [55]:
# we automatically assign a CKD stage when a patient is created
p1.stage

### Generate a care plan

In [56]:
p1.generate_careplan()
print(p1.careplan.plan_name)

Plan: Delay


In [57]:
p1.careplan.list_components()

Plan Components


[('T2D Management', 'Cost Reduction: 0.65')]

### Try it out for a different patient
- This patient has more co-morbidities. Check them out!
- Because of this, we should expect a more comprehensive care plan even though they're still just stage 2

In [58]:
p2.patient_config

{'patient_id': nan,
 't2d': True,
 'depression': True,
 'bmi': True,
 'hypertension': True,
 'smoking': True,
 'dob': nan,
 'race': nan,
 'ethnicity': nan,
 'gender': nan,
 'zip': nan,
 'eGFR': nan}

In [59]:
p2.stage

In [62]:
p2.generate_careplan()
p2.careplan.list_components()

Plan Components


[('Hypertension Management', 'Cost Reduction: 0.5'),
 ('T2D Management', 'Cost Reduction: 0.65'),
 ('Additional Time - Health Equity', 'Cost Reduction: 1'),
 ('Depression Treatment', 'Cost Reduction: 0.8'),
 ('Dietary Program', 'Cost Reduction: 0.7')]